In [3]:
import os

# The specific Xenium path you provided
ROOT_DIR = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/Xenium/zarr_data"

def verify_folder_structure():
    if not os.path.exists(ROOT_DIR):
        print(f"❌ Error: The root directory does not exist: {ROOT_DIR}")
        return

    # Get all subdirectories (Assumed to be sample names)
    samples = sorted([d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))])
    
    print(f"📂 Root Directory: {ROOT_DIR}")
    print(f"🔎 Found {len(samples)} potential sample folders.")
    print("-" * 90)
    # Header for the verification table
    print(f"{'Sample Name':<25} | {'Found HE?':<12} | {'Found MIF?':<12} | {'MIF Mask Exists?':<20}")
    print("-" * 90)
    
    valid_count = 0
    issues = []

    # Check first 5 and last 5 to keep output clean
    for i, sample in enumerate(samples):
        sample_path = os.path.join(ROOT_DIR, sample)
        
        # 1. Explicitly check for the subfolders "he" and "mif"
        he_folder_path = os.path.join(sample_path, "he")
        mif_folder_path = os.path.join(sample_path, "mif")
        
        he_folder_exists = os.path.isdir(he_folder_path)
        mif_folder_exists = os.path.isdir(mif_folder_path)
        
        # 2. Check for the specific mask file we need to copy
        mif_mask_path = os.path.join(mif_folder_path, "nuclei_masks.zarr")
        mif_mask_exists = os.path.exists(mif_mask_path)
        
        # 3. Validation Logic
        if he_folder_exists and mif_folder_exists and mif_mask_exists:
            valid_count += 1
        else:
            issues.append(sample)
        
        # 4. Print Verification (First 5 and Last 5 samples)
        if i < 5 or i >= len(samples) - 5:
            # Create visual status indicators
            he_status = "✅ YES" if he_folder_exists else "❌ NO"
            mif_status = "✅ YES" if mif_folder_exists else "❌ NO"
            mask_status = "✅ YES" if mif_mask_exists else "❌ MISSING"
            
            print(f"[{i+1:>2}] {sample:<20} | {he_status:<12} | {mif_status:<12} | {mask_status:<20}")

    print("-" * 90)
    print(f"📊 Summary:")
    print(f"   Total Folders Scanned: {len(samples)}")
    print(f"   Valid Structure:       {valid_count}")
    
    if issues:
        print(f"   ❌ Issues found in {len(issues)} folders (Missing 'he'/'mif' folder or source mask):")
        print(f"      {issues}")
    else:
        print("   ✅ ALL folders have valid 'he' and 'mif' subfolders.")

if __name__ == "__main__":
    verify_folder_structure()

📂 Root Directory: /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/Xenium/zarr_data
🔎 Found 9 potential sample folders.
------------------------------------------------------------------------------------------
Sample Name               | Found HE?    | Found MIF?   | MIF Mask Exists?    
------------------------------------------------------------------------------------------
[ 1] breast               | ✅ YES        | ✅ YES        | ✅ YES               
[ 2] cervical             | ✅ YES        | ✅ YES        | ✅ YES               
[ 3] lung                 | ✅ YES        | ✅ YES        | ✅ YES               
[ 4] lungV1               | ✅ YES        | ✅ YES        | ✅ YES               
[ 5] lymph_node           | ✅ YES        | ✅ YES        | ✅ YES               
[ 6] ovarian              | ✅ YES        | ✅ YES        | ✅ YES               
[ 7] pancreasV1           | ✅ YES        | ✅ YES        | ✅ YES               
[ 8] prostate             | ✅ YES        | ✅ YES        | 

In [4]:
import os
import shutil
from tqdm import tqdm

# The specific Xenium path you provided
ROOT_DIR = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/Xenium/zarr_data"

def execute_mask_replacement():
    # 1. Get all sample folders
    if not os.path.exists(ROOT_DIR):
        print(f"❌ Error: Root directory not found: {ROOT_DIR}")
        return

    samples = sorted([d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))])
    
    print(f"📂 Target Directory: {ROOT_DIR}")
    print(f"⚠️  WARNING: This will PERMANENTLY DELETE 'he/nuclei_masks.zarr' in {len(samples)} folders")
    print("    and replace it with a copy of 'mif/nuclei_masks.zarr'.")
    print("-" * 60)
    
    # 2. Safety Confirmation
    confirm = input("🔴 Are you sure you want to proceed? (Type 'yes' to continue): ")
    if confirm.lower() != 'yes':
        print("❌ Operation cancelled. No files were touched.")
        return

    print("\n🚀 Starting replacement process...")
    
    success_count = 0
    error_count = 0
    
    for sample in tqdm(samples, desc="Processing Samples"):
        sample_path = os.path.join(ROOT_DIR, sample)
        
        # Define paths
        src_path = os.path.join(sample_path, "mif", "nuclei_masks.zarr")
        dst_path = os.path.join(sample_path, "he", "nuclei_masks.zarr")
        
        # Check source validity
        if not os.path.exists(src_path):
            tqdm.write(f"❌ Skipped {sample}: Source MIF mask missing ({src_path})")
            error_count += 1
            continue
            
        try:
            # A. Delete old H&E mask if it exists
            if os.path.exists(dst_path):
                shutil.rmtree(dst_path)
            
            # B. Copy MIF mask to H&E location
            shutil.copytree(src_path, dst_path)
            success_count += 1
            
        except Exception as e:
            tqdm.write(f"❌ Error on {sample}: {e}")
            error_count += 1

    print("-" * 60)
    print("✅ Operation Complete")
    print(f"   Successfully Swapped: {success_count}")
    print(f"   Errors/Skipped:       {error_count}")

if __name__ == "__main__":
    execute_mask_replacement()

📂 Target Directory: /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/Xenium/zarr_data
⚠️  WARNING: This will PERMANENTLY DELETE 'he/nuclei_masks.zarr' in 9 folders
    and replace it with a copy of 'mif/nuclei_masks.zarr'.
------------------------------------------------------------


🔴 Are you sure you want to proceed? (Type 'yes' to continue):  yes



🚀 Starting replacement process...


Processing Samples: 100%|██████████| 9/9 [01:09<00:00,  7.68s/it]

------------------------------------------------------------
✅ Operation Complete
   Successfully Swapped: 9
   Errors/Skipped:       0


In [5]:
import os

# The specific CRC path you provided
ROOT_DIR = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/ORION-CRC/zarr_data"

def verify_crc_structure():
    if not os.path.exists(ROOT_DIR):
        print(f"❌ Error: The root directory does not exist: {ROOT_DIR}")
        return

    # Get all subdirectories (Assumed to be sample names like CRC01, CRC02...)
    samples = sorted([d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))])
    
    print(f"📂 Root Directory: {ROOT_DIR}")
    print(f"🔎 Found {len(samples)} potential sample folders.")
    print("-" * 90)
    # Header for the verification table
    print(f"{'Sample Name':<25} | {'Found HE?':<12} | {'Found MIF?':<12} | {'MIF Mask Exists?':<20}")
    print("-" * 90)
    
    valid_count = 0
    issues = []

    # Check first 5 and last 5 to keep output clean
    for i, sample in enumerate(samples):
        sample_path = os.path.join(ROOT_DIR, sample)
        
        # 1. Explicitly check for the subfolders "he" and "mif"
        he_folder_path = os.path.join(sample_path, "he")
        mif_folder_path = os.path.join(sample_path, "mif")
        
        he_folder_exists = os.path.isdir(he_folder_path)
        mif_folder_exists = os.path.isdir(mif_folder_path)
        
        # 2. Check for the specific mask file we need to copy
        mif_mask_path = os.path.join(mif_folder_path, "nuclei_masks.zarr")
        mif_mask_exists = os.path.exists(mif_mask_path)
        
        # 3. Validation Logic
        if he_folder_exists and mif_folder_exists and mif_mask_exists:
            valid_count += 1
        else:
            issues.append(sample)
        
        # 4. Print Verification (First 5 and Last 5 samples)
        if i < 5 or i >= len(samples) - 5:
            # Create visual status indicators
            he_status = "✅ YES" if he_folder_exists else "❌ NO"
            mif_status = "✅ YES" if mif_folder_exists else "❌ NO"
            mask_status = "✅ YES" if mif_mask_exists else "❌ MISSING"
            
            print(f"[{i+1:>2}] {sample:<20} | {he_status:<12} | {mif_status:<12} | {mask_status:<20}")

    print("-" * 90)
    print(f"📊 Summary:")
    print(f"   Total Folders Scanned: {len(samples)}")
    print(f"   Valid Structure:       {valid_count}")
    
    if issues:
        print(f"   ❌ Issues found in {len(issues)} folders:")
        print(f"      {issues}")
    else:
        print("   ✅ ALL folders have valid 'he' and 'mif' subfolders.")

if __name__ == "__main__":
    verify_crc_structure()

📂 Root Directory: /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/ORION-CRC/zarr_data
🔎 Found 41 potential sample folders.
------------------------------------------------------------------------------------------
Sample Name               | Found HE?    | Found MIF?   | MIF Mask Exists?    
------------------------------------------------------------------------------------------
[ 1] CRC01                | ✅ YES        | ✅ YES        | ✅ YES               
[ 2] CRC02                | ✅ YES        | ✅ YES        | ✅ YES               
[ 3] CRC03                | ✅ YES        | ✅ YES        | ✅ YES               
[ 4] CRC04                | ✅ YES        | ✅ YES        | ✅ YES               
[ 5] CRC05                | ✅ YES        | ✅ YES        | ✅ YES               
[37] CRC36                | ✅ YES        | ✅ YES        | ✅ YES               
[38] CRC37                | ✅ YES        | ✅ YES        | ✅ YES               
[39] CRC38                | ✅ YES        | ✅ YES      

In [6]:
import os
import shutil
from tqdm import tqdm

# The specific CRC path you verified
ROOT_DIR = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/ORION-CRC/zarr_data"

def execute_crc_mask_replacement():
    # 1. Get all sample folders
    if not os.path.exists(ROOT_DIR):
        print(f"❌ Error: Root directory not found: {ROOT_DIR}")
        return

    samples = sorted([d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))])
    
    print(f"📂 Target Directory: {ROOT_DIR}")
    print(f"⚠️  WARNING: This will PERMANENTLY DELETE 'he/nuclei_masks.zarr' in {len(samples)} folders")
    print("    and replace it with a copy of 'mif/nuclei_masks.zarr'.")
    print("-" * 60)
    
    # 2. Safety Confirmation
    confirm = input("🔴 Are you sure you want to proceed? (Type 'yes' to continue): ")
    if confirm.lower() != 'yes':
        print("❌ Operation cancelled. No files were touched.")
        return

    print("\n🚀 Starting replacement process...")
    
    success_count = 0
    error_count = 0
    
    for sample in tqdm(samples, desc="Processing CRC Samples"):
        sample_path = os.path.join(ROOT_DIR, sample)
        
        # Define paths
        src_path = os.path.join(sample_path, "mif", "nuclei_masks.zarr")
        dst_path = os.path.join(sample_path, "he", "nuclei_masks.zarr")
        
        # Check source validity
        if not os.path.exists(src_path):
            tqdm.write(f"❌ Skipped {sample}: Source MIF mask missing ({src_path})")
            error_count += 1
            continue
            
        try:
            # A. Delete old H&E mask if it exists
            if os.path.exists(dst_path):
                shutil.rmtree(dst_path)
            
            # B. Copy MIF mask to H&E location
            shutil.copytree(src_path, dst_path)
            success_count += 1
            
        except Exception as e:
            tqdm.write(f"❌ Error on {sample}: {e}")
            error_count += 1

    print("-" * 60)
    print("✅ Operation Complete")
    print(f"   Successfully Swapped: {success_count}")
    print(f"   Errors/Skipped:       {error_count}")

if __name__ == "__main__":
    execute_crc_mask_replacement()

📂 Target Directory: /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/ORION-CRC/zarr_data
⚠️  WARNING: This will PERMANENTLY DELETE 'he/nuclei_masks.zarr' in 41 folders
    and replace it with a copy of 'mif/nuclei_masks.zarr'.
------------------------------------------------------------


🔴 Are you sure you want to proceed? (Type 'yes' to continue):  yes



🚀 Starting replacement process...


Processing CRC Samples: 100%|██████████| 41/41 [02:08<00:00,  3.12s/it]

------------------------------------------------------------
✅ Operation Complete
   Successfully Swapped: 41
   Errors/Skipped:       0


In [7]:
import json

file_path = "/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/inference_dsp/xenium_results_twochannel_cells/cell_segmentation.geojson"

with open(file_path, 'r') as f:
    data = json.load(f)

# The number of cells is the length of the 'features' list
cell_count = len(data['features'])
print(f"Total cells: {cell_count}")

Total cells: 1101345
